# 데이터 전처리를 실험하는 파일
@creator: 김기범

# import

In [90]:
import pandas as pd

In [91]:
train_df = pd.read_csv('../dataset/train/train.csv')
test_df = pd.read_csv('../dataset/test/test_data.csv')

train_df.shape, test_df.shape

((32470, 6), (7765, 6))

In [92]:
train_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [93]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32470 entries, 0 to 32469
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              32470 non-null  int64 
 1   sentence        32470 non-null  object
 2   subject_entity  32470 non-null  object
 3   object_entity   32470 non-null  object
 4   label           32470 non-null  object
 5   source          32470 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


## 1. Entity 분리하기

In [94]:
for column in train_df.columns:
    value = train_df.iloc[0][column]
    print(column, ':', value, type(value))

id : 0 <class 'numpy.int64'>
sentence : 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다. <class 'str'>
subject_entity : {'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'} <class 'str'>
object_entity : {'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'} <class 'str'>
label : no_relation <class 'str'>
source : wikipedia <class 'str'>


In [95]:
eval(train_df.iloc[0]['subject_entity'])

{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}

In [96]:
for type_entity in ['subject', 'object']:
    column = f"{type_entity}_entity"

    word_list, type_list = [], []
    start_idx_list, end_idx_list = [], []
    
    for i in range(len(train_df)):
        dictionary = eval(train_df.iloc[i][column])

        word_list.append(dictionary['word'])
        start_idx_list.append(dictionary['start_idx'])
        end_idx_list.append(dictionary['end_idx'])
        type_list.append(dictionary['type'])
    
    train_df[column] = word_list
    for key in ['start_idx', 'end_idx', 'type']:
        train_df[f"{type_entity}_{key}"] = eval(f"{key}_list")

train_df.head()

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,no_relation,wikipedia,24,26,ORG,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,no_relation,wikitree,19,23,ORG,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,org:member_of,wikitree,21,24,ORG,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,org:top_members/employees,wikitree,13,17,ORG,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,no_relation,wikipedia,22,30,ORG,0,3,DAT


# 2. 중복 데이터 처리

In [97]:
train_df.shape

(32470, 12)

In [98]:
train_df.drop(train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity', 'label'], keep='last')].index, axis=0).shape

(32424, 12)

In [99]:
train_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first').shape

(32424, 12)

In [100]:
label_key = {
    key: 30-idx for idx, key in enumerate(train_df['label'].value_counts().index)
}

label_key

{'no_relation': 30,
 'org:top_members/employees': 29,
 'per:employee_of': 28,
 'per:title': 27,
 'org:member_of': 26,
 'org:alternate_names': 25,
 'per:origin': 24,
 'org:place_of_headquarters': 23,
 'per:date_of_birth': 22,
 'per:alternate_names': 21,
 'per:spouse': 20,
 'per:colleagues': 19,
 'per:parents': 18,
 'org:founded': 17,
 'org:members': 16,
 'per:date_of_death': 15,
 'org:product': 14,
 'per:children': 13,
 'per:place_of_residence': 12,
 'per:other_family': 11,
 'per:place_of_birth': 10,
 'org:founded_by': 9,
 'per:product': 8,
 'per:siblings': 7,
 'org:political/religious_affiliation': 6,
 'per:religion': 5,
 'per:schools_attended': 4,
 'org:dissolved': 3,
 'org:number_of_employees/members': 2,
 'per:place_of_death': 1}

In [101]:
case_2_df = train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep=False)].drop(train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity', 'label'], keep=False)].index, axis=0)
case_2_df['sort'] = case_2_df['label'].apply(lambda x: label_key[x])
case_2_df.sort_values(by='sort', inplace=True)
case_2_df

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
25094,25094,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:place_of_headquarters,wikitree,3,6,ORG,26,28,LOC,23
3296,3296,"이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악...",강태환,색소폰,per:title,wikitree,62,64,PER,58,60,POH,27
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29
6749,6749,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,no_relation,wikipedia,0,3,ORG,9,11,PER,30
8364,8364,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,no_relation,wikitree,15,17,PER,3,5,PER,30


In [102]:
case_2_df[case_2_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep='first')]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
25094,25094,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:place_of_headquarters,wikitree,3,6,ORG,26,28,LOC,23
6749,6749,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,no_relation,wikipedia,0,3,ORG,9,11,PER,30
8364,8364,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,no_relation,wikitree,15,17,PER,3,5,PER,30
10320,10320,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,no_relation,wikitree,10,12,PER,52,57,ORG,30
18458,18458,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,no_relation,wikipedia,44,47,ORG,54,56,PER,30
22258,22258,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,no_relation,wikitree,16,18,PER,24,26,LOC,30


In [103]:
case_2_df[case_2_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep='last')]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29


In [104]:
case_2_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity'], keep='first')

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
3296,3296,"이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악...",강태환,색소폰,per:title,wikitree,62,64,PER,58,60,POH,27
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29


In [105]:
view_df = train_df.drop(case_2_df.index, axis=0) # case 2에 해당하는 데이터 삭제
case_2_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity'], keep='first', inplace=True)
new_df = pd.concat([view_df, case_2_df[case_2_df.columns[:-1]]], axis=0)

train_df.shape, view_df.shape, new_df.shape

((32470, 12), (32457, 12), (32464, 12))

In [106]:
new_df.columns

Index(['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source',
       'subject_start_idx', 'subject_end_idx', 'subject_type',
       'object_start_idx', 'object_end_idx', 'object_type'],
      dtype='object')

# 3. 특수 태그 달기

- entity 사이에 [ENT] [/ENT] 추가
- 일본어와 한자는 정규식 이용해서 태그 추가 ([JAP], [CHN])

## 3.1 entity 사이에 [ENT] 태그 추가

In [107]:
for _, row in train_df.iterrows():
    sentence = row['sentence']
    print(sentence)
    for check, idx in enumerate(sorted([row['subject_start_idx'], row['subject_end_idx'], row['object_start_idx'], row['object_end_idx']], reverse=True)):
        if check % 2 == 0:
            sentence = sentence[:idx+1] + " [\ENT] " + sentence[idx+1:]
        else:
            sentence = sentence[:idx] + "[ENT] " + sentence[idx:]
    print(sentence)
    break
print("Done.")

〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
〈Something〉는 [ENT] 조지 해리슨 [\ENT] 이 쓰고 [ENT] 비틀즈 [\ENT] 가 1969년 앨범 《Abbey Road》에 담은 노래다.
Done.


### 3.1.1 entity 사이에 [{S|O}:{type}] 태그 추가

In [121]:
for _, row in train_df.iterrows():
    print("="*5, "subject", "="*5)
    print(row['subject_entity'], "-", row['subject_type'])
    print("="*5, "object", "="*5)
    print(row['object_entity'], "-", row['object_type'])

    sentence = row['sentence']
    print(sentence)
    trigger = True if row['object_end_idx'] > row['subject_end_idx'] else False
    for check, idx in enumerate(sorted([row['subject_start_idx'], row['subject_end_idx'], row['object_start_idx'], row['object_end_idx']], reverse=True)):
        if trigger:
            token = f"O:{row['object_type']}"
        else:
            token = f"S:{row['subject_type']}"

        if check % 2 == 0:
            sentence = sentence[:idx+1] + f" [\{token}] " + sentence[idx+1:]
        else:
            sentence = sentence[:idx] + f"[{token}] " + sentence[idx:]
            trigger = not trigger
    print(sentence)
    break
print("Done.")

===== subject =====
비틀즈 - ORG
===== object =====
조지 해리슨 - PER
〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
〈Something〉는 [O:PER] 조지 해리슨 [\O:PER] 이 쓰고 [S:ORG] 비틀즈 [\S:ORG] 가 1969년 앨범 《Abbey Road》에 담은 노래다.
Done.


## 3.2 [JAP], [CHN] 태그 달아주기

In [112]:
jap_idx = train_df['sentence'].str.extract('([ぁ-ゔァ-ヴー々〆〤一-龥]+)').dropna().index

jap_idx, len(jap_idx)

(Int64Index([  590,  1991,  2175,  2269,  2713,  2725,  3949,  4380,  4964,
              5520,  5529,  6706,  7040,  7602,  8848,  9717, 10419, 11054,
             11491, 12146, 12150, 12447, 12484, 12491, 13781, 14181, 14771,
             15423, 15791, 16002, 16143, 17289, 19469, 20474, 20869, 20932,
             20994, 21658, 21729, 23138, 23144, 23625, 23877, 25151, 26533,
             27327, 28449, 29072, 30003, 30499, 31075, 31439, 31491, 31531],
            dtype='int64'),
 54)

In [119]:
view_df = train_df.copy(deep=True)
view_df['sentence'].replace(r'[ぁ-ゔァ-ヴー々〆〤一-龥]+', '[OTH]', regex=True, inplace=True)

view_df.loc[jap_idx]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type
590,590,일본의 진언종은 구카이([OTH]홍법 대사: 774-835)가 중국 당나라의 장안으...,구카이,진언종,per:religion,wikipedia,9,11,PER,4,6,ORG
1991,1991,"미즈하라 코요미(보통 요미([OTH])([OTH])로 불린다, 한국어 더빙판은 박재...",미즈하라 코요미,요미(よみ),per:alternate_names,wikipedia,0,7,PER,12,17,PER
2175,2175,《베요네타》([OTH])는 일본의 Platinum Games와 세가가 개발한 액션 ...,Platinum Games,베요네타,org:product,wikipedia,19,32,ORG,1,4,POH
2269,2269,시라토리 닌자부로([OTH] [OTH])는 아오야마 고쇼의 만화 《명탐정 코난》에 ...,시라토리 닌자부로,아오야마 고쇼,no_relation,wikipedia,0,8,PER,24,30,PER
2713,2713,"그렇게 에렌과 그를 열렬히 신봉하고 추종하는 예거파 병사들, 신병들의 병단의 손아귀...",헌병단,나일 도크,org:top_members/employees,wikipedia,215,217,ORG,223,227,PER
2725,2725,"""호시조라 미유키""([OTH] 한국명:김다솜)는 토에이 애니메이션 제작의 애니메이션...",미유키,みゆき,per:alternate_names,wikipedia,6,8,PER,11,13,PER
3949,3949,자회사인 주식회사 애니메이션 두([OTH])는 교토 애니메이션의 제작 부서로서 설립...,애니메이션 두,株式会社アニメーションドゥウ,org:alternate_names,wikipedia,10,16,ORG,18,31,ORG
4380,4380,배우 카라타 에리카([OTH] [OTH]·22)와의 불륜으로 모두를 충격에 빠트린 ...,히가시데 마사히로,東出昌大,per:alternate_names,wikitree,41,49,PER,51,54,PER
4964,4964,"이 조직은 일본사회당 및 그 후신인 사회민주당을 지지했지만, 민주당 결성 이후 지지...",일본사회당,사회민주당,org:alternate_names,wikipedia,6,10,ORG,20,24,ORG
5520,5520,"소설가 아쿠타가와 류노스케가 쓴 두 편의 단편 소설, 1915년작 〈라쇼몽〉([OT...",아쿠타가와 류노스케,소설가,per:title,wikipedia,4,13,PER,0,2,POH


# 4. 스폐셜 토큰 추가

In [122]:
arr = ['[OTH]', '[ENT]', '[/ENT]']

# entity detail
for category in ['S', 'O']:
    for type in ['PER', 'NOH', 'ORG', 'LOC', 'POH', 'DUR', 'PNT', 'TIM', 'MNY', 'DAT']:
        arr.append(f"[{category}:{type}]")
        arr.append(f"[/{category}:{type}]")
    
arr, len(arr)

(['[OTH]',
  '[ENT]',
  '[/ENT]',
  '[S:PER]',
  '[/S:PER]',
  '[S:NOH]',
  '[/S:NOH]',
  '[S:ORG]',
  '[/S:ORG]',
  '[S:LOC]',
  '[/S:LOC]',
  '[S:POH]',
  '[/S:POH]',
  '[S:DUR]',
  '[/S:DUR]',
  '[S:PNT]',
  '[/S:PNT]',
  '[S:TIM]',
  '[/S:TIM]',
  '[S:MNY]',
  '[/S:MNY]',
  '[S:DAT]',
  '[/S:DAT]',
  '[O:PER]',
  '[/O:PER]',
  '[O:NOH]',
  '[/O:NOH]',
  '[O:ORG]',
  '[/O:ORG]',
  '[O:LOC]',
  '[/O:LOC]',
  '[O:POH]',
  '[/O:POH]',
  '[O:DUR]',
  '[/O:DUR]',
  '[O:PNT]',
  '[/O:PNT]',
  '[O:TIM]',
  '[/O:TIM]',
  '[O:MNY]',
  '[/O:MNY]',
  '[O:DAT]',
  '[/O:DAT]'],
 43)